In [21]:
import numpy as np
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets
import plotly.graph_objs as go
from ipywidgets import interact

npz_dict = [
    '../output/camera/leaving-pr-57/output_3D/output_keypoints_3d.npz',
    '../output/camera/leaving-pr-55/output_3D/output_keypoints_3d.npz',
    '../output/camera/leaving-pr-54/output_3D/output_keypoints_3d.npz',
    '../output/camera/leaving-pr-53/output_3D/output_keypoints_3d.npz',
    '../output/camera/leaving-pr-52/output_3D/output_keypoints_3d.npz',
    '../output/camera/leaving-pr-51/output_3D/output_keypoints_3d.npz',
]

redata_index = [0,1,2,5]
all_data = {}
start_frame = 100
for index, npz in enumerate(npz_dict):      # Load the data from the provided .npz file
    data = np.load(npz)
    # Inspect the shape and contents of the 'reconstruction' dataset
    if index in redata_index:
        reconstruction_data = data['reconstruction'][start_frame:]
    else:
        reconstruction_data = data['reconstruction']
    
    print(reconstruction_data.shape)
    all_data[index] = reconstruction_data



(77, 17, 3)
(86, 17, 3)
(79, 17, 3)
(56, 17, 3)
(65, 17, 3)
(83, 17, 3)


In [22]:
# 骨骼连接数据
connections = [
    [0, 1], [1, 2], [2, 3], [0, 4], [4, 5],
    [5, 6], [0, 7], [7, 8], [8, 9], [9, 10],
    [8, 11], [11, 12], [12, 13], [8, 14], [14, 15], [15, 16]
]
# 点的标签


In [23]:
def calculate_angle_between_vectors(v1, v2):
    """Calculate the angle in degrees between two vectors."""
    unit_vector_1 = v1 / np.linalg.norm(v1)
    unit_vector_2 = v2 / np.linalg.norm(v2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)
    return np.degrees(angle)


# 添加角度显示到图形中
def add_angle_display(fig, pos1, pos2, pos3, label):

    vector1 = pos2 - pos1
    vector2 = pos2 - pos3
    angle = calculate_angle_between_vectors(vector1, vector2)
    angle_text = f"{angle:.2f}°"

    fig.add_trace(go.Scatter3d(
        x=[pos2[0]],  
        y=[pos2[2]],  # 适应您的轴设置
        z=[-pos2[1]],
        text=[angle_text],
        mode='text',
        textfont=dict(size=20),
        textposition="top center"
    ))

        # 计算中点  
    midpoint1 = (pos1 + pos2) / 2  
    midpoint2 = (pos2 + pos3) / 2  
    
    # 生成链接曲线  
    t = np.linspace(0, 1, 100)  
    curve_x = (1 - t) * midpoint1[0] + t * midpoint2[0]  
    curve_y = (1 - t) * midpoint1[2] + t * midpoint2[2]  # 适应您的轴设置  
    curve_z = -((1 - t) * midpoint1[1] + t * midpoint2[1])  
    
    fig.add_trace(go.Scatter3d(  
        x=curve_x,  
        y=curve_y,  
        z=curve_z,  
        mode='lines',  
        line=dict(color='green', width=2)  
    ))  

# 添加角度显示到图形中
def add_angle_displayV2(fig, pos1, pos2, vector2, label):
    vector1 = pos2 - pos1
    vector2 = vector2
    angle = calculate_angle_between_vectors(vector1, vector2)
    angle_text = f"{angle:.2f}°"
    fig.add_trace(go.Scatter3d(
        x=[pos2[0]],  
        y=[pos2[2]],  # 适应您的轴设置
        z=[-pos2[1]],
        text=[angle_text],
        mode='text',
        textfont=dict(size=20),
        textposition="top center"
    ))

def get_joint_position(data, index):  
    return data[index]  

In [24]:
# 初始化FigureWidget
# 初始化FigureWidget并设置尺寸  
# 初始化FigureWidget并设置尺寸（使用数字而不是字符串）  
fig = go.FigureWidget(  
    layout={  
        'width': 1000,  # 可以调整宽度，例如800, 1000等  
        'height': 1500   # 可以调整高度，例如600, 800等  
    }  
)  

# 设置布局
fig.update_layout(scene=dict(
                    xaxis=dict(title='X'),
                    yaxis=dict(title='Y'),  # 确保y轴垂直方向
                    zaxis=dict(title='Z')
                ))


labels = ['Hips', 'RightThigh', 'RightShin', 'RightFoot', 'LeftThigh', 'LeftShin', 'LeftFoot', 'Spine', 'Neck', 'UpperNeck', 'TopHead', 'LeftShoulder', 'LeftArm', 'LeftHand', 'RightShoulder', 'RightArm', 'RightHand']
# 确保颜色被定义为字符串形式的rgb  
colors_dict = {  
    0: 'rgb(255, 0, 0)',    # 红色  
    1: 'rgb(255, 165, 0)',  # 橙色  
    2: 'rgb(255, 255, 0)',  # 黄色  
    3: 'rgb(0, 128, 0)',    # 绿色  
    4: 'rgb(0, 0, 255)',    # 蓝色  
    5: 'rgb(128, 0, 128)'   # 紫色  
}  

In [25]:
import numpy as np  

def align_left_right_thigh_per_frame(data, labels):  
    """  
    Align the LeftThigh-RightThigh line to the x-axis direction for each frame.  
    
    :param data: The skeleton dataset for all frames, a numpy array of shape (n_frames, n_joints, 3).  
    :param labels: List of joint labels to identify the indices for LeftThigh and RightThigh.  
    :return: The aligned dataset for each frame.  
    """  
    # 找到 'LeftThigh' 和 'RightThigh' 的索引  
    left_thigh_index = labels.index('LeftThigh')  
    right_thigh_index = labels.index('RightThigh')  

    # 对每一帧的数据进行对齐  
    for frame in range(data.shape[0]):  
        # 计算大腿向量  
        thigh_vector = data[frame, left_thigh_index] - data[frame, right_thigh_index]  
        # 标准化大腿向量  
        thigh_vector /= np.linalg.norm(thigh_vector)  

        # 目标 x 轴方向  
        target_vector_x = np.array([1, 0, 0])  
        
        # 计算旋转轴（即大腿向量与目标向量的叉积）  
        axis = np.cross(thigh_vector, target_vector_x)  
        
        # 如果轴的模量非常小，意味着已经对齐或几乎对齐  
        if np.linalg.norm(axis) > 1e-6:  
            # 归一化旋转轴  
            axis /= np.linalg.norm(axis)  
            # 计算旋转角度  
            angle = np.arccos(np.clip(np.dot(thigh_vector, target_vector_x), -1.0, 1.0))  
            # 计算旋转矩阵  
            K = np.array([[0, -axis[2], axis[1]], [axis[2], 0, -axis[0]], [-axis[1], axis[0], 0]])  
            rotation_matrix = np.eye(3) + np.sin(angle) * K + (1 - np.cos(angle)) * np.dot(K, K)  
            
            # 应用旋转矩阵到当前帧的骨架数据  
            data[frame] = np.dot(data[frame], rotation_matrix.T)  

    return data

In [26]:
# 确保定义了每个关节对应的标签列表  
labels = ['Hips', 'RightThigh', 'RightShin', 'RightFoot', 'LeftThigh', 'LeftShin', 'LeftFoot', 'Spine', 'Neck', 'UpperNeck', 'TopHead', 'LeftShoulder', 'LeftArm', 'LeftHand', 'RightShoulder', 'RightArm', 'RightHand']


# 对每个数据集进行对齐  
aligned_data = {key: align_left_right_thigh_per_frame(data, labels) for key, data in all_data.items()}  

print("Alignment complete. Ready for further visualization or analysis.")

Alignment complete. Ready for further visualization or analysis.


In [27]:
print(aligned_data[0].shape)

(77, 17, 3)


In [28]:
all_data = aligned_data

In [29]:
import plotly.graph_objects as go  
import numpy as np  
import ipywidgets as widgets  
from IPython.display import display  

# 假设相关的数据和函数已定义，包括 npz_dict, colors, labels, get_joint_position。  

# 初始化图形  
fig = go.FigureWidget(  
    layout={  
        'width': 1000,  # 可以调整宽度，例如800, 1000等  
        'height': 1000   # 可以调整高度，例如600, 800等  
    }  
)  

# 更新你的 display_3D 函数，确保颜色作为字符串传递给 plotly  
def display_3D(reconstruction_data, show_color, visible=True, index=0):  
    frame_data = reconstruction_data[index]  
    for connection in connections:  
        x = [frame_data[connection[0], 0], frame_data[connection[1], 0]]  
        y = [frame_data[connection[0], 2], frame_data[connection[1], 2]]  
        z = [-frame_data[connection[0], 1], -frame_data[connection[1], 1]]  
        fig.add_trace(go.Scatter3d(  
            x=x, y=y, z=z,   
            mode='lines',   
            line=dict(color=show_color),   
            showlegend=False  
        ))  
    
    for i, label in enumerate(labels):  
        fig.add_trace(go.Scatter3d(  
            x=[frame_data[i, 0]],  
            y=[frame_data[i, 2]],  
            z=[-frame_data[i, 1]],  
            mode='markers',  
            marker=dict(size=10, color=show_color),  
            name=label,  
        ))  

    # 示例：计算并显示大腿和小腿的角度
    right_thigh_pos = get_joint_position(frame_data, labels.index('RightThigh'))
    right_shin_pos = get_joint_position(frame_data, labels.index('RightShin'))
    right_foot_pos = get_joint_position(frame_data, labels.index('RightFoot'))
    
    add_angle_display(fig, right_thigh_pos, right_shin_pos, right_foot_pos, "Right Leg")

    # 示例：计算并显示大腿和小腿的角度
    Hips_pos = get_joint_position(frame_data, labels.index('Hips'))
    Head_pos = get_joint_position(frame_data, labels.index('Neck'))
    vertical_vector =  np.array([0, 1, 0])  # 取默认的垂直向量 
    
    add_angle_displayV2(fig, Head_pos, Hips_pos, vertical_vector, "Body")
    
# 清空图形数据的函数  
def clear_figure():  
    fig.data = []  


# Apply alignment to all datasets  
# aligned_data = [align_left_right_thigh_per_frame(data, labels) for data in all_data]  

# 按钮的回调函数  
# 继续使用到 update_display 函数会使用这些颜色  
def update_display(change):  
    clear_figure()  
    selected_indices = [i for i, toggle in enumerate(toggles) if toggle.value]  
    for index in selected_indices:  
        display_3D(all_data[index], show_color=colors_dict[index], visible=True, index=slider.value)  

# 创建每个数据集的ToggleButtons  
toggles = [widgets.ToggleButton(value=True, description=f'Data {i+1}', button_style='', layout=widgets.Layout(width='100px')) for i in range(len(npz_dict))]  

# 为每个按钮添加观察者  
for toggle in toggles:  
    toggle.observe(update_display, names='value')  

# 创建滑块  
slider = widgets.IntSlider(min=0, max=reconstruction_data.shape[0] - 1, step=1, value=0)  
slider.observe(update_display, names='value')  

# 水平排列的ToggleButtons  
toggle_box = widgets.HBox(toggles)  

# 显示所有小部件和图形  
display(toggle_box, slider, fig)  

# 执行初始渲染  
update_display(None)

IntSlider(value=0, max=82)

FigureWidget({
    'data': [], 'layout': {'height': 1000, 'template': '...', 'width': 1000}
})